<a href="https://colab.research.google.com/github/sung429/DeepLearning/blob/master/pytorch_simpleRNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

In [0]:
n_hidden = 35
lr = 0.01
epochs = 1000

In [0]:
string = "hello pytorch. how long can a rnn cell remember?"
chars = "abcdefghijklmnopqrstuvwxyz ?!.,:;01"
char_list = [i for i in chars]
n_letters = len(char_list)

In [0]:
# 원핫 벡터로 변환
def string_to_onehot(string):
  start = np.zeros(shape=len(char_list), dtype=int)
  end = np.zeros(shape=len(char_list), dtype=int)
  start[-2] = 1
  end[-1] = 1
  for i in string:
    idx = char_list.index(i)
    zero = np.zeros(shape=n_letters, dtype=int)
    zero[idx]=1
    start = np.vstack([start, zero])
  output = np.vstack([start, end])
  return output

In [0]:
def onehot_to_word(onehot_1):
  onehot = torch.Tensor.numpy(onehot_1)
  return char_list[onehot.argmax()]

In [0]:
class RNN(nn.Module):
  def __init__(self, input_size, hidden_size, output_size):
    super(RNN, self).__init__()

    self.input_size = input_size
    self.hidden_size = hidden_size
    self.output_size = output_size

    self.i2h = nn.Linear(input_size, hidden_size)
    self.h2h = nn.Linear(hidden_size, hidden_size)
    self.i2o = nn.Linear(hidden_size, output_size)
    self.act_fn = nn.Tanh()

  def forward(self, input, hidden):
    hidden = self.act_fn(self.i2h(input)+self.h2h(hidden))
    output = self.i2o(hidden)
    return output, hidden

  def init_hidden(self):
    return torch.zeros(1, self.hidden_size)

In [0]:
rnn = RNN(n_letters, n_hidden, n_letters)

In [0]:
loss_func = nn.MSELoss()
optimizer = torch.optim.Adam(rnn.parameters(), lr=lr)

In [9]:
one_hot = torch.from_numpy(string_to_onehot(string)).type_as(torch.FloatTensor())

for i in range(epochs):
  rnn.zero_grad()
  total_loss = 0
  hidden = rnn.init_hidden()

  for j in range(one_hot.size()[0]-1):
    input_ = one_hot[j:j+1,:]
    target = one_hot[j+1]

    output, hidden = rnn.forward(input_, hidden)
    loss = loss_func(output.view(-1), target.view(-1))
    total_loss += loss
    input_ = output

  total_loss.backward()
  optimizer.step()

  if i % 10  == 0:
    print(total_loss)

tensor(2.5972, grad_fn=<AddBackward0>)
tensor(0.9801, grad_fn=<AddBackward0>)
tensor(0.6594, grad_fn=<AddBackward0>)
tensor(0.4487, grad_fn=<AddBackward0>)
tensor(0.3038, grad_fn=<AddBackward0>)
tensor(0.2293, grad_fn=<AddBackward0>)
tensor(0.1713, grad_fn=<AddBackward0>)
tensor(0.1444, grad_fn=<AddBackward0>)
tensor(0.1242, grad_fn=<AddBackward0>)
tensor(0.0964, grad_fn=<AddBackward0>)
tensor(0.0824, grad_fn=<AddBackward0>)
tensor(0.0713, grad_fn=<AddBackward0>)
tensor(0.0729, grad_fn=<AddBackward0>)
tensor(0.0603, grad_fn=<AddBackward0>)
tensor(0.0523, grad_fn=<AddBackward0>)
tensor(0.0469, grad_fn=<AddBackward0>)
tensor(0.0509, grad_fn=<AddBackward0>)
tensor(0.0457, grad_fn=<AddBackward0>)
tensor(0.0386, grad_fn=<AddBackward0>)
tensor(0.0356, grad_fn=<AddBackward0>)
tensor(0.0414, grad_fn=<AddBackward0>)
tensor(0.0328, grad_fn=<AddBackward0>)
tensor(0.0296, grad_fn=<AddBackward0>)
tensor(0.0277, grad_fn=<AddBackward0>)
tensor(0.0351, grad_fn=<AddBackward0>)
tensor(0.0260, grad_fn=<A

In [10]:
start = torch.zeros(1, len(char_list))
start[:,-2] = 1

with torch.no_grad():
  hidden = rnn.init_hidden()
  input_ = start
  output_string = ""
  for i in range(len(string)):
    output, hidden = rnn.forward(input_, hidden)
    output_string += onehot_to_word(output.data)
    input_ = output

print(output_string)

hello pye rne rnng rnn  rnnhernng rnnh rnne lrnn
